In [2]:
import pandas as pd

In [1]:
df = pd.read_csv('power.csv')

print(df.head())

NameError: name 'pd' is not defined

In [4]:
# Imprimiendo el OBS_FLAG.
print(df['OBS_FLAG'])

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
23135      z
23136      z
23137      z
23138      z
23139      z
Name: OBS_FLAG, Length: 23140, dtype: object
